<a href="https://colab.research.google.com/github/xavdx/Nasa-NEO/blob/main/Anshav_NASA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [13]:
import pandas as pd

# **1) 10,000 records:-**

In [14]:
API_KEY="eqVatllvuCpxiEFspRsnO9de3yDEsddcVoplX881"

import requests
from datetime import datetime
url= f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-07&api_key={API_KEY}"

asteroids_data=[]
target=10000
while len(asteroids_data)<target:
    response=requests.get(url)
    data=response.json()
    details=data['near_earth_objects']

    for date,asteroid_details in details.items():
        for ast in asteroid_details:
            approach_data = ast['close_approach_data'][0]

            asteroids_data.append({
                'id': int(ast['id']),
                'neo_reference_id': int(ast['neo_reference_id']),
                'name': ast['name'],
                'magnitude': float(ast['absolute_magnitude_h']),
                'estimated_diameter_min_km': float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
                'estimated_diameter_max_km': float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
                'potentially_hazardous': ast['is_potentially_hazardous_asteroid'],
                'close_approach_date': datetime.strptime(approach_data['close_approach_date'], '%Y-%m-%d'),
                'relative_velocity_kmph': float(approach_data['relative_velocity']['kilometers_per_hour']),
                'astronomical_dist': float(approach_data['miss_distance']['astronomical']),
                'miss_distance_km': float(approach_data['miss_distance']['kilometers']),
                'miss_distance_lunar': float(approach_data['miss_distance']['lunar']),
                'orbiting_body': approach_data['orbiting_body']
            })

            if len(asteroids_data) >= target:
                break
        if len(asteroids_data) >= target:
            break
    url=data['links']['next']

In [15]:
#Display the list
asteroids_data

[{'id': 2415949,
  'neo_reference_id': 2415949,
  'name': '415949 (2001 XY10)',
  'magnitude': 19.37,
  'estimated_diameter_min_km': 0.3552670883,
  'estimated_diameter_max_km': 0.7944013596,
  'potentially_hazardous': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity_kmph': 57205.8951204341,
  'astronomical_dist': 0.3372535274,
  'miss_distance_km': 50452409.349026635,
  'miss_distance_lunar': 131.1916221586,
  'orbiting_body': 'Earth'},
 {'id': 3160747,
  'neo_reference_id': 3160747,
  'name': '(2003 SR84)',
  'magnitude': 26.0,
  'estimated_diameter_min_km': 0.0167708462,
  'estimated_diameter_max_km': 0.0375007522,
  'potentially_hazardous': False,
  'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
  'relative_velocity_kmph': 38589.054833182,
  'astronomical_dist': 0.1323425924,
  'miss_distance_km': 19798169.933318187,
  'miss_distance_lunar': 51.4812684436,
  'orbiting_body': 'Earth'},
 {'id': 3309828,
  'neo_reference_id': 33098

In [16]:
#length of list
len(asteroids_data)

10000

In [17]:
#checking all required fields are present or not
asteroids_data[0]

{'id': 2415949,
 'neo_reference_id': 2415949,
 'name': '415949 (2001 XY10)',
 'magnitude': 19.37,
 'estimated_diameter_min_km': 0.3552670883,
 'estimated_diameter_max_km': 0.7944013596,
 'potentially_hazardous': False,
 'close_approach_date': datetime.datetime(2024, 1, 2, 0, 0),
 'relative_velocity_kmph': 57205.8951204341,
 'astronomical_dist': 0.3372535274,
 'miss_distance_km': 50452409.349026635,
 'miss_distance_lunar': 131.1916221586,
 'orbiting_body': 'Earth'}

# **2) SQL Connection with Table Creation:-**

In [19]:
#Database, cursor and table creation.
import sqlite3
conn = sqlite3.connect('nasa.db')
cursor = conn.cursor()

cursor.execute("""CREATE TABLE asteroids (
    id INT,
    name TEXT,
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN)""")

cursor.execute("""CREATE TABLE close_approach (
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body TEXT)""")

conn.commit()

OperationalError: table asteroids already exists

In [20]:
#insertion of records
conn = sqlite3.connect('nasa.db')
cursor = conn.cursor()

insert_asteroids_query = """
INSERT INTO asteroids (
    id, name, absolute_magnitude_h,
    estimated_diameter_min_km, estimated_diameter_max_km,
    is_potentially_hazardous_asteroid
) VALUES (?, ?, ?, ?, ?, ?)
"""

for ast in asteroids_data:
    values = (
        ast['id'],
        ast['name'],
        ast['magnitude'],
        ast['estimated_diameter_min_km'],
        ast['estimated_diameter_max_km'],
        ast['potentially_hazardous']
    )
    cursor.execute(insert_asteroids_query, values)

conn.commit()
conn.close()

In [21]:
conn = sqlite3.connect('nasa.db')
cursor = conn.cursor()

insert_close_approach_query = """
INSERT INTO close_approach (
    neo_reference_id, close_approach_date, relative_velocity_kmph,
    astronomical, miss_distance_km, miss_distance_lunar, orbiting_body
) VALUES (?, ?, ?, ?, ?, ?, ?)
"""

for ast in asteroids_data:
    approach_values = (
        ast['neo_reference_id'],
        ast['close_approach_date'].strftime('%Y-%m-%d'),
        ast['relative_velocity_kmph'],
        ast['astronomical_dist'],
        ast['miss_distance_km'],
        ast['miss_distance_lunar'],
        ast['orbiting_body']
    )
    cursor.execute(insert_close_approach_query, approach_values)

conn.commit()
conn.close()

In [5]:
!wget -q -O - ipv4.icanhazip.com

34.16.151.132


In [29]:
!pip install streamlit -q
!npm install -g localtunnel

with open("app.py", "w") as f:
    f.write('''import streamlit as st
import sqlite3
import pandas as pd

conn = sqlite3.connect('nasa.db')

st.set_page_config(page_title="NASA NEO Dashboard", layout="wide")
st.title("NASA Near-Earth Object (NEO) Dashboard")
st.markdown("Explore asteroid approaches to Earth using public NASA data.")

# Sidebar for predefined queries
st.sidebar.header("🚀Predefined SQL Queries")
queries = {
    "1. Count asteroid approaches": """
        SELECT neo_reference_id, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY approach_count DESC;
    """,
    "2. Average velocity of each asteroid": """
        SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
        FROM close_approach
        GROUP BY neo_reference_id
        ORDER BY avg_velocity DESC;
    """,
    "3. Top 10 fastest asteroids": """
        SELECT ca.neo_reference_id, a.name, MAX(ca.relative_velocity_kmph) AS max_velocity
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        GROUP BY ca.neo_reference_id
        ORDER BY max_velocity DESC
        LIMIT 10;
    """,
    "4. Hazardous asteroids > 3 approaches": """
        SELECT a.id, a.name, COUNT(*) AS approach_count
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE a.is_potentially_hazardous_asteroid = 1
        GROUP BY a.id
        HAVING approach_count > 3;
    """,
    "5. Month with most asteroid approaches": """
        SELECT strftime('%m-%Y', close_approach_date) AS month, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY month
        ORDER BY approach_count DESC
        LIMIT 1;
    """,
    "6. Asteroid with fastest approach": """
        SELECT ca.neo_reference_id, a.name, ca.relative_velocity_kmph
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        ORDER BY ca.relative_velocity_kmph DESC
        LIMIT 1;
    """,
    "7. Sort by estimated diameter": """
        SELECT id, name, estimated_diameter_max_km
        FROM asteroids
        ORDER BY estimated_diameter_max_km DESC;
    """,
    "8. Closest approaches over time": """
        SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.miss_distance_km
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        ORDER BY ca.neo_reference_id, ca.close_approach_date;
    """,
    "9. Closest approach per asteroid": """
        SELECT ca.neo_reference_id, a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS min_distance
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        GROUP BY ca.neo_reference_id;
    """,
    "10. Velocity > 50,000 km/h": """
        SELECT DISTINCT a.name, ca.relative_velocity_kmph
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.relative_velocity_kmph > 50000;
    """,
    "11. Count of approaches per month": """
        SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS approach_count
        FROM close_approach
        GROUP BY month;
    """,
    "12. Highest brightness asteroid": """
        SELECT id, name, absolute_magnitude_h
        FROM asteroids
        ORDER BY absolute_magnitude_h ASC
        LIMIT 1;
    """,
    "13. Hazardous vs non-hazardous count": """
        SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
        FROM asteroids
        GROUP BY is_potentially_hazardous_asteroid;
    """,
    "14. Passed closer than Moon": """
        SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.miss_distance_lunar
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.miss_distance_lunar < 1;
    """,
    "15. Within 0.05 AU": """
        SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.astronomical
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        WHERE ca.astronomical < 0.05;
    """,
    "16. Top 5 closest miss distances": """
        SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.miss_distance_km
        FROM close_approach ca
        JOIN asteroids a ON ca.neo_reference_id = a.id
        ORDER BY ca.miss_distance_km ASC
        LIMIT 5;
    """,
    "17. Largest asteroid by diameter range": """
        SELECT id, name, estimated_diameter_max_km - estimated_diameter_min_km AS diameter_range
        FROM asteroids
        ORDER BY diameter_range DESC
        LIMIT 5;
    """,
    "18. Average brightness of hazardous vs non-hazardous": """
        SELECT is_potentially_hazardous_asteroid, AVG(absolute_magnitude_h) AS avg_brightness
        FROM asteroids
        GROUP BY is_potentially_hazardous_asteroid;
    """,
    "19. Total unique asteroids": """
        SELECT COUNT(DISTINCT id) AS unique_asteroids FROM asteroids;
    """,
    "20. Total number of close approaches": """
        SELECT COUNT(*) AS total_approaches FROM close_approach;
    """,
    "21. Average miss distance (AU)": """
        SELECT AVG(astronomical) AS avg_miss_distance_AU FROM close_approach;
    """,
    "22. Count of approaches by orbiting body": """
        SELECT orbiting_body, COUNT(*) AS count
        FROM close_approach
        GROUP BY orbiting_body
        ORDER BY count DESC;
    """
}

selected_query = st.sidebar.selectbox("Select a query to run", list(queries.keys()))
if selected_query:
    st.subheader(f"📊 Result for: {selected_query}")
    df = pd.read_sql_query(queries[selected_query], conn)
    st.dataframe(df, use_container_width=True)

st.sidebar.header("🔧Filter Data Interactively")
st.subheader("🎯Custom Asteroid Filter")

date_range = st.sidebar.date_input("Select close approach date range",
                                   value=(pd.to_datetime("2024-01-01"), pd.to_datetime("2025-04-13")))

min_velocity, max_velocity = st.sidebar.slider("Velocity (km/h)", 0.0, 165000.00, (0.0, 50000.0), step=500.0)
min_diameter, max_diameter = st.sidebar.slider("Estimated Diameter (km)", 0.0, 10.30, (0.0, 2.5), step=0.1)
min_au, max_au = st.sidebar.slider("Astronomical Distance (AU)", 0.0, 1.0, (0.0, 0.5), step=0.01)
min_ld, max_ld = st.sidebar.slider("Lunar Distance", 0.0, 100.0, (0.0, 50.0), step=1.0)
hazardous_only = st.sidebar.checkbox("Show only potentially hazardous asteroids", value=False)

query = f"""
    SELECT ca.close_approach_date, a.name, ca.relative_velocity_kmph, ca.miss_distance_km,
           ca.astronomical, ca.miss_distance_lunar,
           a.estimated_diameter_min_km, a.estimated_diameter_max_km,
           a.is_potentially_hazardous_asteroid
    FROM close_approach ca
    JOIN asteroids a ON ca.neo_reference_id = a.id
    WHERE date(ca.close_approach_date) BETWEEN '{date_range[0]}' AND '{date_range[1]}'
      AND ca.relative_velocity_kmph BETWEEN {min_velocity} AND {max_velocity}
      AND ca.astronomical BETWEEN {min_au} AND {max_au}
      AND ca.miss_distance_lunar BETWEEN {min_ld} AND {max_ld}
      AND a.estimated_diameter_max_km BETWEEN {min_diameter} AND {max_diameter}
"""

if hazardous_only:
    query += " AND a.is_potentially_hazardous_asteroid = 1"

query += " ORDER BY ca.close_approach_date;"

filtered_df = pd.read_sql_query(query, conn)
st.subheader("📅Filtered Close Approaches")
st.dataframe(filtered_df, use_container_width=True)
''')

!npx localtunnel --port 8501 --subdomain neoasteroids & streamlit run app.py

⠙⠹⠸⠼⠴⠦
changed 22 packages in 811ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦

⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.151.132:8501

⠴⠦your url is: https://neoasteroids.loca.lt
  Stopping...


# **3) Queries:-**

1.
Count how many times each asteroid has approached Earth

In [ ]:
conn = sqlite3.connect('nasa.db')
cursor = conn.cursor()

In [ ]:
q1= """
SELECT neo_reference_id, COUNT(*) AS approach_count
FROM close_approach
GROUP BY neo_reference_id
ORDER BY approach_count DESC;
"""

df = pd.read_sql_query(q1, conn)
df.head()

,neo_reference_id,approach_count
0,54356994,5
1,54161317,5
2,3989284,5
3,3840791,5
4,3771633,5


2. Average velocity of each asteroid over multiple approaches

In [ ]:
query2 = """
SELECT neo_reference_id, AVG(relative_velocity_kmph) AS avg_velocity
FROM close_approach
GROUP BY neo_reference_id
ORDER BY avg_velocity DESC;
"""
df2 = pd.read_sql_query(query2, conn)
df2.head()

,neo_reference_id,avg_velocity
0,54392072,173071.830081
1,3803884,161520.897925
2,3790060,158279.864701
3,3425295,154336.407297
4,3092220,152218.220518


3. Top 10 fastest asteroids

In [ ]:
query3 = """
SELECT ca.neo_reference_id, a.name, MAX(ca.relative_velocity_kmph) AS max_velocity
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY ca.neo_reference_id
ORDER BY max_velocity DESC
LIMIT 10;
"""
df3 = pd.read_sql_query(query3, conn)
df3.head()

,neo_reference_id,name,max_velocity
0,54392072,(2023 TS3),173071.830081
1,3803884,(2018 GG5),161520.897925
2,3790060,(2017 WP28),158279.864701
3,3425295,(2008 QB),154336.407297
4,3092220,(1999 GR6),152218.220518


4. Hazardous asteroids that approached Earth more than 3 times

In [ ]:
Q4 = """
SELECT a.id, a.name, COUNT(*) AS approach_count
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE a.is_potentially_hazardous_asteroid = 1
GROUP BY a.id
HAVING approach_count > 3;
"""
df4 = pd.read_sql_query(Q4, conn)
df4.head()

,id,name,approach_count
0,2002340,2340 Hathor (1976 UA),4
1,2004034,4034 Vishnu (1986 PA),4
2,2065679,65679 (1989 UQ),4
3,2101955,101955 Bennu (1999 RQ36),4
4,2137126,137126 (1999 CF9),4


5. Month with most asteroid approaches

In [ ]:
query5 = """
SELECT strftime('%m-%Y', close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY approach_count DESC
LIMIT 1;
"""
df5 = pd.read_sql_query(query5, conn)
df5

,month,approach_count
0,10-2024,904


6. Asteroid with fastest ever approach speed

In [ ]:
q6 = """
SELECT ca.neo_reference_id, a.name, ca.relative_velocity_kmph
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
ORDER BY ca.relative_velocity_kmph DESC
LIMIT 1;
"""
df6 = pd.read_sql_query(q6, conn)
df6

,neo_reference_id,name,relative_velocity_kmph
0,54392072,(2023 TS3),173071.830081


7. Sort asteroids by maximum estimated diameter

In [ ]:
query7 = """
SELECT id, name, estimated_diameter_max_km
FROM asteroids
ORDER BY estimated_diameter_max_km DESC;
"""
df7 = pd.read_sql_query(query7, conn)
df7.head()

,id,name,estimated_diameter_max_km
0,2000887,887 Alinda (A918 AA),10.281109
1,2001685,1685 Toro (1948 OA),8.280184
2,2066146,66146 (1998 TU3),7.980681
3,2005660,5660 (1974 MA),4.989304
4,2533671,533671 (2014 LJ21),3.926811


8. Asteroids with closest approaches over time

In [ ]:
query8="""
SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.miss_distance_km
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
ORDER BY ca.neo_reference_id, ca.close_approach_date ASC;
"""
df8 = pd.read_sql_query(query8, conn)
df8.head()

,neo_reference_id,name,close_approach_date,miss_distance_km
0,2000887,887 Alinda (A918 AA),2025-01-08,1.229661e+07
1,2001685,1685 Toro (1948 OA),2024-01-20,1.990593e+07
2,2002063,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
3,2002063,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
4,2002063,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07


9. Closest approach per asteroid

In [ ]:
Q9="""SELECT ca.neo_reference_id, a.name, ca.close_approach_date, MIN(ca.miss_distance_km) AS min_distance
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
GROUP BY ca.neo_reference_id;
"""
df9 = pd.read_sql_query(Q9, conn)
df9

,neo_reference_id,name,close_approach_date,min_distance
0,2000887,887 Alinda (A918 AA),2025-01-08,1.229661e+07
1,2001685,1685 Toro (1948 OA),2024-01-20,1.990593e+07
2,2002063,2063 Bacchus (1977 HB),2024-03-31,1.795253e+07
3,2002340,2340 Hathor (1976 UA),2024-11-02,5.948137e+07
4,2004034,4034 Vishnu (1986 PA),2025-04-01,2.345060e+07
...,...,...,...,...
7368,54533317,(2025 KW2),2024-12-07,2.766141e+07
7369,54533318,(2025 KX2),2024-12-08,1.818143e+07
7370,54533499,(2025 JB3),2025-04-03,2.909198e+07
7371,54533502,(2025 KR4),2024-05-12,6.763441e+06


10. Asteroids with velocity > 50,000 km/h

In [ ]:
query10="""
SELECT DISTINCT a.name, ca.relative_velocity_kmph
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.relative_velocity_kmph > 50000
ORDER BY ca.relative_velocity_kmph DESC;
"""
df10 = pd.read_sql_query(query10, conn)
df10

,name,relative_velocity_kmph
0,(2023 TS3),173071.830081
1,(2018 GG5),161520.897925
2,(2017 WP28),158279.864701
3,(2008 QB),154336.407297
4,(1999 GR6),152218.220518
...,...,...
3047,(2025 DQ1),50068.549301
3048,(2019 YB4),50048.281234
3049,(2024 GA2),50047.074620
3050,(2021 GN1),50027.279641


11. Count of asteroid approaches per month

In [ ]:
Q11 = """
SELECT strftime('%Y-%m', close_approach_date) AS month, COUNT(*) AS approach_count
FROM close_approach
GROUP BY month
ORDER BY month;
"""
df11 = pd.read_sql_query(Q11, conn)
df11

,month,approach_count
0,2024-01,646
1,2024-02,607
2,2024-03,655
3,2024-04,709
4,2024-05,578
5,2024-06,526
6,2024-07,469
7,2024-08,621
8,2024-09,853
9,2024-10,904


12. Asteroid with highest brightness(lowest magnitude value)

In [ ]:
query12="""
SELECT id, name, absolute_magnitude_h
FROM asteroids
ORDER BY absolute_magnitude_h ASC
LIMIT 1;
"""
df12 = pd.read_sql_query(query12, conn)
df12

,id,name,absolute_magnitude_h
0,2000887,887 Alinda (A918 AA),13.81


13. Count of hazardous vs non-hazardous asteroids

In [ ]:
que13="""
SELECT is_potentially_hazardous_asteroid, COUNT(*) AS count
FROM asteroids
GROUP BY is_potentially_hazardous_asteroid;
"""
df13 = pd.read_sql_query(que13, conn)
df13

,is_potentially_hazardous_asteroid,count
0,0,9505
1,1,495


14. Asteroids that passed closer than the Moon

In [ ]:
query14="""
SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.miss_distance_lunar
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.miss_distance_lunar < 1;
"""
df14 = pd.read_sql_query(query14, conn)
df14

,neo_reference_id,name,close_approach_date,miss_distance_lunar
0,54418649,(2024 AD),2024-01-04,0.631031
1,54420444,(2024 AG4),2024-01-11,0.885273
2,54420624,(2024 AM4),2024-01-11,0.310675
3,54427461,(2024 BA16),2024-01-15,0.364068
4,54420280,(2024 AZ3),2024-01-14,0.647173
...,...,...,...,...
300,54525109,(2025 FV12),2025-03-28,0.264304
301,54525643,(2025 FM18),2025-04-02,0.721706
302,54525753,(2025 GA),2025-04-02,0.221098
303,54526132,(2025 GS),2025-04-05,0.444223


15. Asteroids that came within 0.05 AU

In [ ]:
q15 ="""
SELECT ca.neo_reference_id, a.name, ca.close_approach_date, ca.astronomical
FROM close_approach ca
JOIN asteroids a ON ca.neo_reference_id = a.id
WHERE ca.astronomical < 0.05;
"""
df15 = pd.read_sql_query(q15, conn)
df15

,neo_reference_id,name,close_approach_date,astronomical
0,3842680,(2019 KK5),2024-01-02,0.026097
1,54417653,(2023 YR),2024-01-02,0.011564
2,54418497,(2024 AA),2024-01-02,0.004081
3,54419485,(2024 AQ1),2024-01-02,0.043829
4,54418438,(2023 YR1),2024-01-01,0.016608
...,...,...,...,...
3615,54380881,(2023 RX1),2025-04-13,0.046395
3616,54525113,(2025 FV13),2025-04-13,0.046640
3617,54525113,(2025 FV13),2025-04-13,0.046640
3618,54526038,(2025 GK),2025-04-13,0.044380


In [ ]:
from google.colab import files
files.download('nasa.db')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>